<a href="https://colab.research.google.com/github/pld000/z3/blob/main/task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install "z3-solver"
from z3 import *

# **Выражения, сорта и декларации**

Задача 1.1: Исследование свойств выражения Создайте выражение expr = (x * y) + 2, где x и y являются переменными типа Int. Выведите его сорт, является ли оно приложением, его декларацию, количество аргументов и каждый аргумент.

In [3]:
x, y = Ints('x y')
expr = (x * y) + 2
print ("Сорт expr:", expr.sort())
print ("Является ли expr приложением:", is_app(expr))
print ("Декларация expr:", expr.decl())
print ("Количество аргументов expr:", expr.num_args())
for i in range(expr.num_args()):
    print (f"Аргумент {i} expr: {expr.arg(i)}")

Сорт expr: Int
Является ли expr приложением: True
Декларация expr: +
Количество аргументов expr: 2
Аргумент 0 expr: x*y
Аргумент 1 expr: 2


1.2
Определить две функции f и g так, чтобы f принимала Int и возвращала Int, а g принимала Int и возвращала Int. Создать выражение n = f(g(x) + 1), где x является Int. Заменить g(x) на y + 5 в n, где y — другая переменная типа Int.

In [4]:
x, y = Ints('x y')
f = Function('f', IntSort(), IntSort())
g = Function('g', IntSort(), IntSort())

n = f(g(x) + 1)
print("Исходное выражение:", n)

substituted_n = substitute(n, (g(x), y + 5))
print("Замещенное выражение:", substituted_n)

Исходное выражение: f(g(x) + 1)
Замещенное выражение: f(y + 5 + 1)


# **2. Массивы**

2.1 Доказательство аксиомы массива с Select и Store Используя prove, продемонстрируйте, что если мы сохраняем значение v по индексу x в массиве A, а затем выбираем значение по индексу x из измененного массива, мы получаем v.



In [5]:
A = Array('A', IntSort(), IntSort())
x, v = Ints('x v')
prove(Select(Store(A, x, v), x) == v)

proved


2.2: Определение и ограничение IntVector Создайте IntVector с именем my_vector с 4 целочисленными элементами. Добавьте ограничение в решатель, что сумма элементов в my_vector должна быть больше или равна 10. Затем проверьте решатель и выведите модель.

In [ ]:
my_vector = IntVector('my_vector', 4)
s = Solver()
s.add(Sum(my_vector) >= 10)
print(s.check())
print(s.model())

# **3. Кванторы**

3.1: Решение для экзистенциально квантифицированной функции Найдите функцию g(x), которая принимает Int и возвращает Int, такую, что существует x, для которого g(x) больше 5.

In [6]:
g = Function('g', IntSort(), IntSort())
x = Int('x')
solve(Exists(x, g(x) > 5))

[g = [else -> 6]]


3.2: Доказательство свойства с использованием универсальных кванторов Докажите, что если f(x) всегда равно x * x, то f(y) всегда равно y * y для любого y.

In [7]:
f = Function('f', IntSort(), IntSort())
x, y = Ints('x y')
prove(Implies(ForAll(x, f(x) == x * x), ForAll(y, f(y) == y * y)))

proved


# **4. Оптимизация**

4.1: Максимизация простой линейной целевой функции Найдите максимальное значение x + y при следующих ограничениях: x >= 0, y >= 0, x + y <= 10 и 2*x + y <= 15.

In [8]:
x, y = Reals('x y')
opt = Optimize()
opt.add(x >= 0, y >= 0)
opt.add(x + y <= 10)
opt.add(2*x + y <= 15)
opt.maximize(x + y)
print(opt.check())
print(opt.model())

sat
[y = 5, x = 5]


4.2: Минимизация функции стоимости с условными выражениями Компания производит два продукта, A и B. Пусть prod_A и prod_B — это произведенные количества. Стоимость производства составляет  5заединицудля𝐴и 7 за единицу для B. Существует специальная скидка: если prod_A меньше 50, ее стоимость за единицу снижается до $4. Общая производственная мощность составляет 200 единиц. Минимизируйте общую стоимость.



In [9]:
prod_A = Int('prod_A')
prod_B = Int('prod_B')
cost = Real('cost')

opt = Optimize()

opt.add(prod_A >= 0, prod_B >= 0)
opt.add(prod_A + prod_B <= 200)

opt.add(cost == If(prod_A < 50, prod_A * 4, prod_A * 5) + prod_B * 7)

opt.minimize(cost)

print(opt.check())
print(opt.model())

sat
[prod_B = 0, prod_A = 0, cost = 0]


# **5. Множественные солверы**

5.1: Передача ограничений между двумя решателями и проверка выполнимости Создайте два решателя: s_alpha и s_beta. Добавьте x > 5 и y < 10 в s_alpha. Затем добавьте ограничения из s_alpha в s_beta и проверьте s_beta на выполнимость.

In [10]:
x, y = Ints('x y')

s_alpha = Solver()
s_alpha.add(x > 5, y < 10)
print("s_alpha до передачи:", s_alpha)

s_beta = Solver()
s_beta.add(s_alpha.assertions())
print("s_beta после передачи:", s_beta)

print("Проверка s_beta:", s_beta.check())
if s_beta.check() == sat:
    print("Модель s_beta:", s_beta.model())

s_alpha до передачи: [x > 5, y < 10]
s_beta после передачи: [x > 5, y < 10]
Проверка s_beta: sat
Модель s_beta: [y = 9, x = 6]


 5.2: Использование нескольких решателей для проверки взаимоисключающих условий Используйте два решателя, s1 и s2, для демонстрации того, что два набора условий являются взаимоисключающими. s1 должен содержать x > 0 и x < 5, а s2 должен содержать x >= 5. Проверьте каждый решатель по отдельности, а затем попытайтесь добавить утверждения одного решателя в другой и покажите, что результат будет unsat.

In [11]:
x = Int('x')

s1 = Solver()
s1.add(x > 0, x < 5)
print("Проверка s1:", s1.check())
if s1.check() == sat:
    print("Модель s1:", s1.model())

s2 = Solver()
s2.add(x >= 5)
print("Проверка s2:", s2.check())
if s2.check() == sat:
    print("Модель s2:", s2.model())

# Попытка объединить взаимоисключающие условия
s1_plus_s2 = Solver()
s1_plus_s2.add(s1.assertions())
s1_plus_s2.add(s2.assertions())
print("Проверка s1 + s2:", s1_plus_s2.check())

Проверка s1: sat
Модель s1: [x = 1]
Проверка s2: sat
Модель s2: [x = 5]
Проверка s1 + s2: unsat
